In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
import csv
import datetime
from numpy import nan

In [ ]:
# В репозитории скрипт, скачивающтий календарь в json. Для упрощения, файл был скачан прямо с гитхаба.
!curl https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/consultant2022.json --output workersdays.json

In [ ]:
f = open('workersdays.json')
data = json.load(f)

In [ ]:
holidays_list = []
if "holidays" in data:
    for i in data['holidays']:
        holidays_list.append(i)

In [ ]:
preholidays_list = []
if "preholidays" in data:
    for i in data['preholidays']:
        preholidays_list.append(i)

In [ ]:
nowork_list = []
if "nowork" in data:
    for i in data['nowork']:
        nowork_list.append(i)

In [ ]:
if (len(holidays_list) > len(preholidays_list)):
       preholidays_list += (len(holidays_list)-len(preholidays_list)) * [nan]
if (len(holidays_list) < len(preholidays_list)):
        holidays_list += (len(preholidays_list)-len(holidays_list)) * [nan]
if (len(holidays_list) > len(nowork_list)):
       nowork_list += (len(holidays_list)-len(nowork_list)) * [nan]

In [ ]:
workers_days = pd.DataFrame({'holidays': holidays_list, 'preholidays': preholidays_list, 'nowork': nowork_list})

In [ ]:
workers_days['preholidays'][2]

In [ ]:
for i in range(workers_days.shape[0]):
    if workers_days['holidays'].any() & ~pd.isnull(workers_days['holidays'][i]):
        workers_days['holidays'][i] = datetime.datetime.strptime(workers_days['holidays'][i], '%Y-%m-%d').date()

    if workers_days['preholidays'].any() & ~pd.isnull(workers_days['preholidays'][i]):
        workers_days['preholidays'][i] = datetime.datetime.strptime(workers_days['preholidays'][i], '%Y-%m-%d').date()

    if workers_days['nowork'].any() & ~pd.isnull(workers_days['nowork'][i]):
         workers_days['nowork'][i] = datetime.datetime.strptime(workers_days['nowork'][i], '%Y-%m-%d').date()

In [ ]:
workers_days

In [ ]:
workers_days.dtypes

In [ ]:
workers_days["holidays"][0]

In [ ]:
workers_days["preholidays"][0]

In [ ]:
dates_ = pd.date_range(start='1/1/2022', end='31/12/2022')
id_ = list(range(1, len(dates_)+1))
df_structure = pd.DataFrame({'id': id_, 'date': dates_, 'if_working': nan, 'in_hours': nan})

In [ ]:
# Создание структуры датасета
df_structure['date'] = df_structure['date'].dt.date

In [ ]:
df_structure

In [ ]:
df_structure.dtypes

In [ ]:
workers_days['nowork'][0]

In [ ]:
df_structure['date'][0]

In [ ]:
for i in range(df_structure.shape[0]):
    if any(workers_days['preholidays'] == df_structure['date'][i]):
        df_structure['if_working'][i] = "7"
        df_structure['in_hours'][i] = 7
    elif any(workers_days['holidays'] == df_structure['date'][i]):
        df_structure['if_working'][i] = "Пр"
        df_structure['in_hours'][i] = 0
    elif any(workers_days['nowork'] == df_structure['date'][i]):
        df_structure['if_working'][i]  = "Пр"
        df_structure['in_hours'][i] = 0
    else:
        df_structure['if_working'][i] = "8"
        df_structure['in_hours'][i] = 8

In [ ]:
df_structure['in_hours'] = df_structure['in_hours'].astype(int)
df_structure['if_working'] = df_structure['if_working']

In [ ]:
df_structure["if_working"].unique()

In [ ]:
print(df_structure.dtypes)

In [ ]:
df_structure.to_csv("workersdays2022.csv", index=False, encoding="utf-8")